In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import re

Here is what the text file looks like. The Information appears to be set up as a dictionary with the categories: ID, ASIN, title, group, salesrank, similar, categories (this looks like it has sub-elements), reviews (again with sub-elements). Actually, it reminds me a bit of an XML file in that way.

Id:   1
ASIN: 0827229534
  title: Patterns of Preaching: A Sermon Sampler
  group: Book
  salesrank: 396585
  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X
  categories: 2
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]
  reviews: total: 2  downloaded: 2  avg rating: 5
    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9
    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5

Id:   2
ASIN: 0738700797
  title: Candlemas: Feast of Flames
  group: Book
  salesrank: 168596
  similar: 5  0738700827  1567184960  1567182836  0738700525  0738700940
  categories: 2
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Earth-Based Religions[12472]|Wicca[12484]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Earth-Based Religions[12472]|Witchcraft[12486]
  reviews: total: 12  downloaded: 12  avg rating: 4.5
    2001-12-16  cutomer: A11NCO6YTE4BTJ  rating: 5  votes:   5  helpful:   4
    2002-1-7  cutomer:  A9CQ3PLRNIR83  rating: 4  votes:   5  helpful:   5
    2002-1-24  cutomer: A13SG9ACZ9O5IM  rating: 5  votes:   8  helpful:   8
    2002-1-28  cutomer: A1BDAI6VEYMAZA  rating: 5  votes:   4  helpful:   4
    2002-2-6  cutomer: A2P6KAWXJ16234  rating: 4  votes:  16  helpful:  16
    2002-2-14  cutomer:  AMACWC3M7PQFR  rating: 4  votes:   5  helpful:   5
    2002-3-23  cutomer: A3GO7UV9XX14D8  rating: 4  votes:   6  helpful:   6
    2002-5-23  cutomer: A1GIL64QK68WKL  rating: 5  votes:   8  helpful:   8
    2003-2-25  cutomer:  AEOBOF2ONQJWV  rating: 5  votes:   8  helpful:   5
    2003-11-25  cutomer: A3IGHTES8ME05L  rating: 5  votes:   5  helpful:   5
    2004-2-11  cutomer: A1CP26N8RHYVVO  rating: 1  votes:  13  helpful:   9
    2005-2-7  cutomer:  ANEIANH0WAT9D  rating: 5  votes:   1  helpful:   1

Id:   3
ASIN: 0486287785
  title: World War II Allied Fighter Planes Trading Cards
  group: Book
  salesrank: 1270652
  similar: 0
  categories: 1
   |Books[283155]|Subjects[1000]|Home & Garden[48]|Crafts & Hobbies[5126]|General[5144]
  reviews: total: 1  downloaded: 1  avg rating: 5
    2003-7-10  cutomer: A3IDGASRQAW8B2  rating: 5  votes:   2  helpful:   2

Id:   4
ASIN: 0842328327
  title: Life Application Bible Commentary: 1 and 2 Timothy and Titus
  group: Book
  salesrank: 631289
  similar: 5  0842328130  0830818138  0842330313  0842328610  0842328572
  categories: 5
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Reference[172810]|Commentaries[12155]|New Testament[12159]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Christian Living[12333]|Discipleship[12335]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Bibles[12059]|Translations[764432]|Life Application[572080]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Bible & Other Sacred Texts[12056]|Bible[764430]|New Testament[572082]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Bibles[12059]|Study Guides, History & Reference[764438]|General[572094]
  reviews: total: 1  downloaded: 1  avg rating: 4
    2004-8-19  cutomer: A2591BUPXCS705  rating: 4  votes:   1  helpful:   1

Ultimately I only need to access the information in the ratings section and the category section. The issue is that both of these are complex sections. The ratings has customer information in it that may actually be useful as a cursory glance shows that some custormers are posting multiple times on the same product - thus skewing the ratings. Further, it has votes and helpfulness aspects and dates. 

The category section is a mess of multiple labels. I'm guessing these are trees in a sense and only the last label matters, but I'm not sure. 

### These are the tasks I need to do:

1. Trustworthiness of ratings
Ratings are susceptible to manipulation, bias etc. What can you say (quantitatively
speaking) about the ratings in this dataset?

So, from the ratings I should make sure to select the customer id (to check for people who are posting multiple reviews on teh same product), the rating, the votes, and the helpful marks. I can look for correations between what rating is given and how helpful it was, recalcualted averages without duplicates, etc. I will also get the line saying what the average rating and total number of ratings is.



2 Category bloat

Consider the product group named 'Books'. Each product in this group is associated with
categories. Naturally, with categorization, there are tradeoffs between how broad or
specific the categories must be.
For this dataset, quantify the following:
a. Is there redundancy in the categorization? How can it be identified/removed?
b. Is is possible to reduce the number of categories drastically (say to 10% of existing
categories) by sacrificing relatively few category entries (say close to 10%)?

I will need to check which category the product is and only keep the books, then make a record of different categories so I can check counts of categories to look for redundencies.

In [2]:
# Define read_large_file()
def read_large_file(file_object):
    """A generator function to read a large file lazily."""

    # Loop indefinitely until the end of the file
    while True:

        # Read a line from the file: data
        data = file_object.readline()

        # Break if this is the end of the file
        if not data:
            break

        # Yield the line of data
        yield data

In [113]:
with open('subset.txt', encoding = 'latin-1') as file:
    print(file.readline())
    print(file.readline())
    print(file.readline())
    a = file.readline()
    row = a.split(':')                    # read the line
    first_col = row[0]                       # select the asin
    if first_col[0] == 'g':
        ID = row[1]
        ID = re.findall(r'\w+', ID)
    print(ID)

Id:   1

ASIN: 0827229534

  title: Patterns of Preaching: A Sermon Sampler

['B00000AU3R']


In [114]:
first_col[2] == 'g' and row[1][1] == 'B'


True

In [95]:
# Initialize an empty dictionary
#Steps to take: 1. extract and save the ID of the entry 2.pull the review information out as lists and parse them to create
# a dataframe 3. append the dataframe to a dictionary with the key being the ID number. 4. Concat the dictionaryies into a 
# multi-level dataframe.
review_dict = {}

# Open a connection to the file
with open('amazon-meta.txt', encoding = 'latin-1') as file:

    # Iterate over the generator from read_large_file()
    for line in read_large_file(file):  
        
        row = line.split(':')                    # read the line
        first_col = row[0]                       # select the asin
        if first_col[0] == 'A':
            ID = row[1]
            ID = re.findall(r'\w+', ID)
            
            df = pd.DataFrame(columns=['CID','rating','votes','helpful']) #create a fresh dataframe associated with this product
        elif line.find("cutomer:") > -1: # getting the ratings
            review_info = line.split()

            customer_idx = str(review_info[2])
            rating = review_info[4]
            votes = review_info[6]
            helpful = review_info[8]
            df = df.append({'CID': customer_idx,'rating':int(rating), 'votes':int(votes),'helpful':int(helpful)},ignore_index = True)
            #This ^ adds a new line to the dataframe with the ratings info
        elif first_col[0] == 'I':
            review_dict[ID[0]] = df
            
        


# Print            
reviews_df = pd.concat(review_dict)
reviews_df.head()

KeyboardInterrupt: 

In [96]:
reviews_df = pd.concat(review_dict)
len(reviews_df)

3271896

In [97]:
reviews_df.to_pickle('first_method_all.pkl')

In [117]:
# Initialize an empty dataframe
#Steps to take: 1. extract and save the ID of the entry 2.pull the review information out as lists and parse them to create
# a dataframe 3. append the dataframe to the dataframe
other_df = pd.DataFrame(columns=['product', 'CID','rating','votes','helpful']) 
cat = 'something'# prepare category check

# Open a connection to the file
with open('amazon-meta.txt', encoding = 'latin-1') as file:

    # Iterate over the generator from read_large_file()
    for line in read_large_file(file):  
        
        row = line.split(':')                    # read the line
        first_col = row[0]                       # select the asin
        if first_col[0] == 'A':
            ID = row[1]
            ID = re.findall(r'\w+', ID)
            
        if len(first_col) >= 3:
            if first_col[2] == 'g' and row[1][1] == 'B':
                cat = 'book'
            elif first_col[2] =='g' and row[1][1] !='B':
                cat = 'something'
                
        if line.find("cutomer:") > -1 and cat == 'book': # getting the ratings
                review_info = line.split()

                customer_idx = str(review_info[2])
                rating = review_info[4]
                votes = review_info[6]
                helpful = review_info[8]
                other_df = other_df.append({'product': ID,'CID': customer_idx,'rating':int(rating), 'votes':int(votes),'helpful':int(helpful)},ignore_index = True)
            #This ^ adds a new line to the dataframe with the ratings info
        
            
        


# Print            
other_df.head()

KeyboardInterrupt: 

In [118]:
len(other_df)

448580

In [ ]:
other_df.to_pickle('second_method.pkl')

In [44]:
review_info

['2004-5-31',
 'cutomer:',
 'ABTSEEYVYQ52M',
 'rating:',
 '5',
 'votes:',
 '9',
 'helpful:',
 '9']

In [42]:
# Here I will again get the ASIN and then collect the category of all the books. 

category_df = pd.DataFrame(columns=['product', 'category']) 
cat = 'something'# prepare category check

with open('subset.txt', encoding = 'latin-1') as file:

    # Iterate over the generator from read_large_file()
    for line in read_large_file(file):  
                row = line.split(':')                    # read the line
        first_col = row[0]                       # select the asin
        if first_col[0] == 'A':
            ID = row[1]
            ID = re.findall(r'\w+', ID)
            
        if len(first_col) >= 3:
            if first_col[2] == 'g' and row[1][1] == 'B':
                cat = 'book'
            elif first_col[2] =='g' and row[1][1] !='B':
                cat = 'something'
        if len(line) >= 3:
            if line[3] == '|' and cat == 'book':
                customer_info = line.split('|')
                category = customer_info[-1][:-2]
                category_df = category_df.append({'product': ID, 'category': category})
            

In [43]:
customer_info
    

['   ',
 'Music[5174]',
 'Styles[301668]',
 'Broadway & Vocalists[265640]',
 'General[912012]\n']

In [ ]:
category = customer_info[-1][:-2]

In [38]:
b[3]

'|'

In [4]:
f = id_dict['Id']

KeyError: 'Id'

In [9]:
f

'   548551\n'

In [12]:
f = re.findall(r'\d+','   548551\n')
f

['548551']

In [11]:
d = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]])
d

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [13]:
di = {} #just practicing making a dictionary of dataframes and then making one multilevel dataframe
di['1']=d
di['2']=d
di

{'1':    0  1  2
 0  1  2  3
 1  4  5  6
 2  7  8  9, '2':    0  1  2
 0  1  2  3
 1  4  5  6
 2  7  8  9}

In [15]:
df = pd.concat(di)
df

0  1  2
1 0  1  2  3
  1  4  5  6
  2  7  8  9
2 0  1  2  3
  1  4  5  6
  2  7  8  9

In [16]:
df[1]

1  0    2
   1    5
   2    8
2  0    2
   1    5
   2    8
Name: 1, dtype: int64

In [18]:
df[1][5]

8